# Notebook [1]: First steps with cdQA

This notebook shows how to use the `cdQA` pipeline to perform question answering on a custom dataset.

***Note:*** *If you are using colab, you will need to install `cdQA` by executing `!pip install cdqa` in a cell.*

In [2]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

/home/ron/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Download pre-trained reader model and example dataset

In [3]:
from cdqa.utils.download import download_model, download_bnpp_data

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')

### Visualize the dataset

In [4]:
df = pd.read_csv('./data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

,date,title,category,link,abstract,paragraphs
0,13.05.2019,The banking jobs : Assistant Vice President – ...,Careers,https://group.bnpparibas/en/news/banking-jobs-...,Within the Group’s Corporate and Institutional...,[I manage a team in charge of designing and im...
1,13.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[With François Hollande, Chairman of French fo..."
2,13.05.2019,"""The bank with an IT budget of more than EUR6 ...",Group,https://group.bnpparibas/en/news/the-bank-budg...,"Interview with Jean-Laurent Bonnafé, Director ...","[We did the groundwork between 2012 and 2016, ..."
3,10.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[As part of the ‘United Tech of Europe’ theme,..."
4,10.05.2019,When Artificial Intelligence participates in r...,Careers,https://group.bnpparibas/en/news/artificial-in...,As the competition to attract talent intensifi...,[Online recruitment is already the norm. Accor...


In [10]:
df.loc[0].paragraphs

['I manage a team in charge of designing and implementing the Enterprise Data Management Framework, which is a set of procedures governing data management. To carry out our mission, we analyze the pertinence of the data in our possession—whether they are financial, transactional, referential, etc.—starting from their origination and capture in our systems until the moment they are consumed for internal and regulatory reports. Our goal is to understand and document the processes that mitigate data risks along its journey, identify potential flaws, to ultimately provide assurance over the data at critical points. It’s a specialization that has grown significantly since the 2008 financial crisis.',
 'With the frameworks that have been and are being implemented, our ability to act on data-related issues is challenged by the sheer volume of data, metadata (i.e. data on data), dashboards and other data visualization tools. It can be difficult to not get overwhelmed when trouble shooting an i

In [9]:
''.join(df.loc[0].paragraphs)

'I manage a team in charge of designing and implementing the Enterprise Data Management Framework, which is a set of procedures governing data management. To carry out our mission, we analyze the pertinence of the data in our possession—whether they are financial, transactional, referential, etc.—starting from their origination and capture in our systems until the moment they are consumed for internal and regulatory reports. Our goal is to understand and document the processes that mitigate data risks along its journey, identify potential flaws, to ultimately provide assurance over the data at critical points. It’s a specialization that has grown significantly since the 2008 financial crisis.With the frameworks that have been and are being implemented, our ability to act on data-related issues is challenged by the sheer volume of data, metadata (i.e. data on data), dashboards and other data visualization tools. It can be difficult to not get overwhelmed when trouble shooting an issue, 

### Instantiate the cdQA pipeline from a pre-trained reader model

In [5]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 901046.84B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

### Execute a query

In [0]:
query = 'Since when does the Excellence Program of BNP Paribas exist?'
prediction = cdqa_pipeline.predict(query)

### Explore predictions

In [14]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: Since when does the Excellence Program of BNP Paribas exist?
answer: January 2016
title: BNP Paribas’ commitment to universities and schools
paragraph: Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.


In [0]:
query2 = "Who's BNP Paribas' CEO?"
prediction2 = cdqa_pipeline.predict(query2)

In [16]:
print('query: {}'.format(query2))
print('answer: {}'.format(prediction2[0]))
print('title: {}'.format(prediction2[1]))
print('paragraph: {}'.format(prediction2[2]))

query: Who's BNP Paribas' CEO?
answer: Jean-Laurent Bonnafé
title: BNP Paribas continues its commitment in favour of integrating refugees in Europe
paragraph: Jean-Laurent Bonnafé, BNP Paribas CEO, declared that, “The refugee drama is a major human catastrophe that is mobilising many organisations and volunteers throughout Europe who are dealing with emergency conditions and providing refugees with the possibility of shelter, work, and a future. BNP Paribas is at their side, not only with financial assistance, but also with help finding employment and, for some, with recruitment. In today’s difficult context, we are committed to pursuing these efforts.”
